In [2]:
import numpy as np
import torch
import torchvision
import torchvision.datasets as datasets
from pytorch_lightning import Trainer
import models
import os
import importlib
import utils
import matplotlib.pyplot as plt
data_dir ="./data"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [35]:
importlib.reload(models)
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\ged11\\Dropbox\\0 TUM\\Deep Learning Seminar\\HousesAndBoats\\utils.py'>

In [7]:
mnist_train = datasets.MNIST(root='./data',train=True,transform=torchvision.transforms.ToTensor(),
                            target_transform=None,download=True)
mnist_test = datasets.MNIST(root = './data', train=False,transform=torchvision.transforms.ToTensor(),
                           target_transform=None,download=True)

In [36]:
model = models.CapsNet(train_dataset=mnist_train,testing_dataset=mnist_test,input_size=[1,28,28],classes=10,routing_iters=3)

28
28


In [ ]:
trainer = Trainer(max_nb_epochs=1, overfit_pct=0.05)
trainer.fit(model)

INFO:root:               Name                 Type Params
0              conv               Conv2d   20 K
1       primaryCaps  PrimaryCapsuleLayer    5 M
2  primaryCaps.conv               Conv2d    5 M
3              relu                 ReLU    0  
Validation sanity check:   0%|                                                                | 0/5 [00:00<?, ?batch/s]

torch.Size([10, 16, 16])


Validation sanity check:  20%|███████████▏                                            | 1/5 [00:01<00:06,  1.58s/batch]

torch.Size([10, 16, 16])


Validation sanity check:  40%|██████████████████████▍                                 | 2/5 [00:01<00:03,  1.23s/batch]

...

Epoch 1:   1%|▍                                    | 2/186 [00:03<06:33,  2.14s/batch, batch_nb=1, loss=2.302, v_nb=16]

torch.Size([10, 16, 16])


Epoch 1:   2%|▌                                    | 3/186 [00:14<14:12,  4.66s/batch, batch_nb=2, loss=2.305, v_nb=16]

torch.Size([10, 16, 16])


Epoch 1:   2%|▊                                    | 4/186 [00:15<11:07,  3.67s/batch, batch_nb=3, loss=2.300, v_nb=16]

In [19]:
image, target = next(iter(mnist_train))

In [20]:
image.shape

torch.Size([1, 28, 28])

In [22]:
target

5

In [34]:
model.forward(image[None,:])

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [33]:
image[None,:].size()

torch.Size([1, 1, 28, 28])